### Laboratorio 9 - Tutorial de Visualziación de Datos

Stefano Aragoni, Carol Arévalo, Luis Santos

---------

*Objetivos*:
- Familiarizarse con el Análisis Exploratorio de Datos (EDA) utilizando Python.
- Aprender a desarrollar y evaluar modelos de aprendizaje automático.
- Reflexionar sobre la aplicabilidad de estas habilidades en contextos profesionales.

*Conjunto de Datos*:
- Utilizaremos datos de la "Encuesta Nacional de Condiciones de Vida 2014" (ENCOVI) de Guatemala. Estos datos deben ser limpiados y preprocesados antes del análisis.

--------

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import Image
import pyreadstat

#### **Parte 1: Análisis Exploratorio de Datos (40 Puntos)**

##### *Carga y Limpieza de Datos (10 puntos):*

Como primer paso, se debe cargar los datos y realizar una limpieza básica. Para esto, se recomienda seguir los siguientes pasos:

- Importar el conjunto de datos.
- Identificar y tratar valores faltantes.
- Normalizar nombres de columnas.

Como primer paso, como se puede observar a continuación, se importa el conjunto de datos y se muestra la información de las columnas. 

En este caso, en ambos archivos, se puede observar una gran cantidad de columnas. Asimismo, inicialmente se puede observar una gran cantidad de valores faltantes. 

In [10]:
# Cargar los datos desde el archivo .sav
df_hogar = pd.read_spss('./Datos_Lab/ENCOVI_Hogar.sav')

# Revisar los primeros registros para entender la estructura de los datos
df_hogar.head()

,REGION,DEPTO,AREA,UPM,NUMHOG,FACTOR,FACTOR3,POBREZA,THOGAR,PPB01,...,P01H10,P01H11,P01H12,P01H13,P01H14,P01H15,P01H16,DIA_ENC,MES_ENC,A_ENC
0,I. METROPOLITANA,Guatemala,Urbana,1.0,1.0,525.0,1575.0,No pobre,3.0,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,AGOSTO,2014.0
1,I. METROPOLITANA,Guatemala,Urbana,1.0,2.0,525.0,1575.0,No pobre,3.0,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,AGOSTO,2014.0
2,I. METROPOLITANA,Guatemala,Urbana,1.0,3.0,525.0,9450.0,Pobre no extr.,18.0,NO,...,Si,Si,Si,Si,Si,Si,Si,22.0,AGOSTO,2014.0
3,I. METROPOLITANA,Guatemala,Urbana,1.0,4.0,525.0,3150.0,Pobre no extr.,6.0,NO,...,No,No,No,No,No,No,No,24.0,AGOSTO,2014.0
4,I. METROPOLITANA,Guatemala,Urbana,1.0,5.0,525.0,1575.0,No pobre,3.0,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,AGOSTO,2014.0


In [12]:
# Cargar los datos desde el archivo .sav
df_personas = pd.read_spss('./Datos_Lab/ENCOVI_Personas.sav')

# Revisar los primeros registros para entender la estructura de los datos
df_hogar.head()

,REGION,DEPTO,AREA,UPM,NUMHOG,FACTOR,FACTOR3,POBREZA,THOGAR,PPB01,...,P01H10,P01H11,P01H12,P01H13,P01H14,P01H15,P01H16,DIA_ENC,MES_ENC,A_ENC
0,I. METROPOLITANA,Guatemala,Urbana,1.0,1.0,525.0,1575.0,No pobre,3.0,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,AGOSTO,2014.0
1,I. METROPOLITANA,Guatemala,Urbana,1.0,2.0,525.0,1575.0,No pobre,3.0,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,AGOSTO,2014.0
2,I. METROPOLITANA,Guatemala,Urbana,1.0,3.0,525.0,9450.0,Pobre no extr.,18.0,NO,...,Si,Si,Si,Si,Si,Si,Si,22.0,AGOSTO,2014.0
3,I. METROPOLITANA,Guatemala,Urbana,1.0,4.0,525.0,3150.0,Pobre no extr.,6.0,NO,...,No,No,No,No,No,No,No,24.0,AGOSTO,2014.0
4,I. METROPOLITANA,Guatemala,Urbana,1.0,5.0,525.0,1575.0,No pobre,3.0,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,AGOSTO,2014.0
